In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import networkit
import networkx

edgeFile = "wikispeedia_paths-and-graph/links.tsv"
categoryFile = "wikispeedia_paths-and-graph/categories.tsv"

## Load category data
Read data from file into Pandas dataframe. This tells us how many nodes there are. The categories are not used for community detection, but we'll use them later to inspect the communities.

In [ ]:
nodeData = pd.read_csv(
    categoryFile,
    names = ["article", "category"],
    skiprows = 13,
    delimiter = '\t'
)
nodeIds = {row[1].article:row[0] for row in nodeData.iterrows()}
nodeData["level_1_category"] = [_.split(".")[1] for _ in nodeData.category.values]
nodeData["level_2_category"] = [
    _.split(".")[2]
    if len(_.split(".")) > 2 else  _.split(".")[1]
    for _ in nodeData.category.values
]
print(len(nodeData), "nodes")
nodeData.head()

## Load link data
For this demo, let's treat the links as undirected. An alternative would be to only include bidirectional links, or find a community detection method that uses edge direction.

In [ ]:
graph = networkit.graph.Graph(len(nodeData))
nxGraph = networkx.Graph()
with open(edgeFile) as f:
    for line in f:
        if not line.startswith('#') and line.strip():
            splitLine = line.strip().split("\t")
            sourceId = nodeIds.get(splitLine[0], None)
            targetId = nodeIds.get(splitLine[1], None)
            if sourceId != None and targetId != None:
                graph.addEdge(sourceId, targetId)
                nxGraph.add_edge(sourceId, targetId)
print(len(graph.edges()), "edges")

## Connected components
List the sizes of the 10 largest weakly connected components.

In [ ]:
componentSizes = list(networkit.components.ConnectedComponents(graph)
                  .run()
                  .getComponentSizes()
                  .values())
componentSizes.sort(key = lambda x: -x)
componentSizes[:10]

## Community detection
Run community detection using the Parallel Louvain Method (PLM). This outputs a `Partition` object.

In [ ]:
plmPartition = networkit.community.PLM(graph).run().getPartition()

In [ ]:
plmPartition.getSubsetIds()

In [ ]:
plmPartition.getMembers(3)

In [ ]:
subsetSizes = {k:v for k,v in plmPartition.subsetSizeMap().items() if v > 1}
print(len(subsetSizes), "communities with more than one member")
subsetSizes

## Inspect the communities
To help us make sense of the communities, let's plot the top article categories for each community.

In [ ]:
nodeData["community"] = plmPartition.getVector()
nodeData.head()

In [ ]:
%matplotlib inline
def plot_categories(
        community = None,
        column = "level_1_category",
        top_n = 10,
        axes = None):
    if axes == None:
        axes = plt.axes()
    if community is not None:
        plotData = nodeData[nodeData.community == community]
    else:
        plotData = nodeData
    (plotData
     .groupby(column)
     .count()
     .sort("article", ascending=False)
     .article
     .head(top_n)
     .plot(kind="bar", ax=axes))
    plt.ylabel("number of articles")

In [ ]:
print(len(nodeData.level_1_category.unique()), "parent categories")
nodeData.level_1_category.unique()

In [ ]:
print(len(nodeData.level_2_category.unique()), "child categories")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,4))
plot_categories(top_n=15, column="level_1_category", axes=ax1)
plot_categories(top_n=15, column="level_2_category", axes=ax2)

In [ ]:
for community in subsetSizes:
    print("\nCommunity %d: %d articles" % (community, subsetSizes[community]))
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,4))
    plot_categories(community, top_n=15, column="level_1_category", axes=ax1)
    plot_categories(community, top_n=15, column="level_2_category", axes=ax2)
    plt.show()

## Export graph to file
Save the graph data, including the communities we found, in a format that can be read with Gephi for visualisation.

In [ ]:
# Add node data to the NetworkX version of the graph
for index,data in nodeData.iterrows():
    nxGraph.node[index] = data
    
# Find the largest connected component
largestCC = nxGraph.subgraph(
    networkit.components.ConnectedComponents(graph)
    .run()
    .getPartition()
    .getMembers(1)
)

# Write the graph data to a file
networkx.write_graphml(largestCC, "wiki.graphml")